In [5]:
import pandas as pd

In [17]:
PATH_test = "/content/test_spam.csv"
PATH_train = "/content/train_spam.csv"

In [18]:
df_test = pd.read_csv(PATH_test)
df_train = pd.read_csv(PATH_train)

print("\n" + "Тренировочный датасет с флагами, является ли сообщение спамом или нет")
display(df_train.head())
print(2 * "\n" + "Тестовый датасет без флагов")
df_test.head()


Тренировочный датасет с флагами, является ли сообщение спамом или нет


,text_type,text
0,ham,make sure alex knows his birthday is over in f...
1,ham,a resume for john lavorato thanks vince i will...
2,spam,plzz visit my website moviesgodml to get all m...
3,spam,urgent your mobile number has been awarded wit...
4,ham,overview of hr associates analyst project per ...




Тестовый датасет без флагов


,text
0,j jim whitehead ejw cse ucsc edu writes j you ...
1,original message from bitbitch magnesium net p...
2,java for managers vince durasoft who just taug...
3,there is a youtuber name saiman says
4,underpriced issue with high return on equity t...


# Base analytics

In [61]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)
nltk.download('wordnet', quiet=True)
from nltk.corpus import stopwords as sw
from nltk.stem import WordNetLemmatizer
from nltk import FreqDist

from sklearn.feature_extraction.text import CountVectorizer

import re
from typing import List

In [62]:
# посмотрим, сколько сообщений каждого типа содержится в тренировочном наборе данных
df_train['text_type'].value_counts()

text_type
ham     11469
spam     4809
Name: count, dtype: int64

In [68]:
def preprocess(text: str) -> List[str]:
    '''
      Функция выполняет серию преобразований для предварительной обработки текста

      Args:

          text - строка текста

      Returns:

          Cписок лемм
    '''
    text = re.sub(r'[^\w\s]', '', text)
    text = text.lower()
    text = word_tokenize(text)
    filtered_tokens = [token for token in text if not token in stopwords and
                                     not token.isdigit() and not len(token) <=3]
    text = [lemmatizer .lemmatize(word) for word in filtered_tokens]

    return text

Для обработки текста будет использоваться лемматизация, а именно Мы можем рассматривать лемматизацию как более сложную версию стемминга. Она приводит каждое слово к его надлежащей базовой форме, то есть к слову, которое мы можем найти в словаре.
WordNet  —  это лексикологическая база, то есть словарь английского языка, созданный специально для обработки естественного языка.

In [69]:
lemmatizer = WordNetLemmatizer()
stopwords = set(sw.words('english'))
# Применим функцию к train и test данным
df_train_tokenized = df_train.copy()
df_train_tokenized['text'] = df_train_tokenized['text'].apply(preprocess)

df_test_tokenized = df_test.copy()
df_test_tokenized['text'] = df_test_tokenized['text'].apply(preprocess)

In [ ]:
most_common_train = [word for sublist in df_train_tokenized['text'] for word in sublist]
most_common_test = [word for sublist in df_test_tokenized['text'] for word in sublist]

# Считаем частоту слов и берем топ 10 для train
freq_dist_train = FreqDist(most_common_train)
top_words_train = freq_dist_train.most_common(10)

# Считаем частоту слов и берем топ 10 для test
freq_dist_test = FreqDist(most_common_test)
top_words_test = freq_dist_test.most_common(10)

# Выводим топы
# train
print("Топ 10 самых часто используемых слов в сообщениях train:")
for word, frequency in top_words_train:
    print(f"{word}: {frequency}")
print("\nТоп 10 самых менее используемых слов в сообщениях train:")
for word, frequency in freq_dist_train.most_common()[:-11:-1]:
    print(f"{word}: {frequency}")
# test
print("\nТоп 10 самых часто используемых слов в сообщениях test:")
for word, frequency in top_words_test:
    print(f"{word}: {frequency}")
print("\nТоп 10 самых менее используемых слов в сообщениях test:")
for word, frequency in freq_dist_test.most_common()[:-11:-1]:
    print(f"{word}: {frequency}")

Топ 10 самых часто используемых слов в сообщениях train:
enron: 4848
vince: 4526
kaminski: 2365
please: 2358
subject: 2203
time: 2041
would: 2036
like: 1843
know: 1706
thanks: 1558

Топ 10 самых менее используемых слов в сообщениях train:
boyz: 1
funfilled: 1
pundit: 1
jawdropping: 1
raunchy: 1
cleavageshowing: 1
bitsy: 1
itsy: 1
balans: 1
saysvidya: 1

Топ 10 самых часто используемых слов в сообщениях test:
enron: 1202
vince: 1042
please: 606
kaminski: 564
would: 555
time: 544
subject: 519
like: 470
know: 452
message: 407

Топ 10 самых менее используемых слов в сообщениях test:
spea: 1
hopeu: 1
texd: 1
nitw: 1
hellogorgeous: 1
planted: 1
aisi: 1
laanat: 1
kyahusbandaur: 1
shoutedtell: 1


In [ ]:
# Разделим сообщения со спамом и без на разные датафреймы
df_spam = df_train_tokenized[df_train_tokenized['text_type'] == 'spam']
df_non_spam = df_train_tokenized[df_train_tokenized['text_type'] == 'ham']

In [ ]:
# Объединяем все токены в один список для спама и не спама
spam_tokens_flat = [word for sublist in df_spam['text'] for word in sublist]
non_spam_tokens_flat = [word for sublist in df_non_spam['text'] for word in sublist]

# Считаем частоту слов и берем топ 10 для спама
freq_dist_spam = FreqDist(spam_tokens_flat)
most_common_spam = freq_dist_spam.most_common(10)

# Считаем частоту слов и берем топ 10 для не спама
freq_dist_non_spam = FreqDist(non_spam_tokens_flat)
most_common_non_spam = freq_dist_non_spam.most_common(10)

# Выводим наиболее часто употребляемые слова
print("Топ 10 самых часто используемых слов в сообщениях со спамом:")
for word, frequency in most_common_spam:
    print(f"{word}: {frequency}")

print("\nТоп 10 самых часто используемых слов в сообщениях без спама:")
for word, frequency in most_common_non_spam:
    print(f"{word}: {frequency}")

Топ 10 самых часто используемых слов в сообщениях со спамом:
free: 1081
call: 566
click: 557
link: 541
account: 502
offer: 485
company: 480
money: 476
time: 472
please: 470

Топ 10 самых часто используемых слов в сообщениях без спама:
enron: 4848
vince: 4525
kaminski: 2365
subject: 2113
please: 1888
would: 1849
time: 1569
like: 1440
thanks: 1407
know: 1389


In [ ]:
# Посмотрим, какие пары слов чаще всего встречаются вместе в спаме
vectorizer = CountVectorizer(ngram_range=(2,2))
df_spam['text'] = df_spam['text'].apply(lambda x: ' '.join(x))
sparse_matrix = vectorizer.fit_transform(df_spam['text'])
frequencies = sum(sparse_matrix).toarray()[0]
ngrams = pd.DataFrame(frequencies, index=vectorizer.get_feature_names_out(),
                                                          columns=['frequency'])
ngrams = ngrams.sort_values(by='frequency', ascending=False)
# выведем частоту больше 100
ngrams[ngrams['frequency'] >= 100]

,frequency
hyperlink hyperlink,154
click link,120
would like,109


In [ ]:
# Какие пармы слов встречаются чаще всего в не спаме
df_non_spam['text'] = df_non_spam['text'].apply(lambda x: ' '.join(x))
sparse_matrix = vectorizer.fit_transform(df_non_spam['text'])
frequencies = sum(sparse_matrix).toarray()[0]
ngrams = pd.DataFrame(frequencies, index=vectorizer.get_feature_names_out(),
                                                          columns=['frequency'])
ngrams = ngrams.sort_values(by='frequency', ascending=False)
# выведем частоту больше 400
ngrams[ngrams['frequency'] >= 400]

,frequency
vince kaminski,2046
shirley crenshaw,526
would like,523
enron enron,509
kaminski subject,471
enron subject,452
kaminski enron,409


## LDA

In [ ]:
!pip install pyLDAvis

In [16]:
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import pyLDAvis
import pyLDAvis.gensim_models
import matplotlib.pyplot as plt
%matplotlib inline

from tqdm import tqdm
from pprint import pprint

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.simplefilter("ignore")

In [ ]:
global coherence_list, model_list

coherence_list = []
model_list = []

In [ ]:
def remove_stopwords(texts: List[str]) -> List[List[str]]:
    """
    Удаляет стоп-слова

    Args:

        texts : Список строк, где каждая строка представляет собой один документ.

    Returns:

        Список документов, представленный в виде списков слов без стоп-слов.
    """
    return [[word for word in simple_preprocess(str(doc)) if word not in stopwords] for doc in texts]

def make_bigrams(texts: List[List[str]]) -> List[List[str]]:
    """
    Преобразует список текстов в список текстов с биграммами

    Args:
        texts : Список документов, представленных в виде списков слов.

    Returns:

        Список документов с добавленными биграммами
    """
    return [bigram_mod[doc] for doc in texts]

def find_model(start: int, finish: int, data_words: list):
  """
    Определяет оптимальное количество тем для LDA модели на основе переданных данных

    Функция итерируется по заданному диапазону чисел, обучает LDA модель для каждого количества тем и
    вычисляет perplexity и coherence для каждой модели. Оптимальное количество тем
    следует выбирать между минимальным perplexity и максимальным coherence

    Args:

        start : Начальное значение диапазона количества тем для оценки

        finish : Конечное значение диапазона количества тем для оценки

        data_words : Список текстов, где каждый текст представлен списком слов

    Returns:

       Число тем с минимальным perplexity и число тем с максимальным coherence
  """
  perplexity_min = float('inf') # Чтобы найти минимальный perplexity, начальное значение присвоим большим
  coherence_max = 0.5           # Ищется максимальная согласованность, а значение обычно начинаетя с чисел, ближе к 1
  theme_amount_p = 0            # Номер темы с минимальным perplexity
  theme_amount_c = 0            # Номер темы с максимальным coherence

  for elem in tqdm(range(start, finish)):
    # Обучение LDA модели
    lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                                id2word=id2word,
                                                num_topics=elem,
                                                random_state=100,
                                                update_every=1,
                                                chunksize=100,
                                                passes=10,
                                                alpha='auto',
                                                per_word_topics=True)
    # Расчет согласованности
    coherence_model_lda = CoherenceModel(model=lda_model, texts=data_words,
                                        dictionary=id2word, coherence='c_v')
    coherence_lda = coherence_model_lda.get_coherence()
    # Поиск минимумального значеня perplexity и максимума согласованности
    if lda_model.log_perplexity(corpus) < perplexity_min:
      perplexity_min = lda_model.log_perplexity(corpus)
      theme_amount_p = elem
    if coherence_lda > coherence_max:
      coherence_max = coherence_lda
      theme_amount_c = elem
    # Сохранение значений согласованности и моделей в принципе
    coherence_list.append(coherence_lda)
    model_list.append(lda_model)
    # Вывод текущего состояния для последуюбщего определения лучшего кол-ва тем
    print(f'{elem}: Perplexity = {round(lda_model.log_perplexity(corpus), 3)}. Coherence Score = {round(coherence_lda, 3)}')
  # Вывод результатов
  print(f'\n\nPerplexity of {theme_amount_p+1} themes: {round(perplexity_min, 3)}' + '\n' +
        f'Coherence Score of {theme_amount_c+1} themes: {round(coherence_max, 3)}\n')

### Spam themes

In [ ]:
# Создаёт модель для обнаружения биграмм
# min_count - отсекает биграммы, которые появляются реже, чем 3 раза
bigram = gensim.models.Phrases(spam_tokens_flat, min_count=3, threshold=100)
bigram_mod = gensim.models.phrases.Phraser(bigram)

In [ ]:
# Удаление стоп-слов
data_words_nostops = remove_stopwords(spam_tokens_flat)
# Создание биграмм из списка слов без стоп-слов
data_words_bigrams = make_bigrams(data_words_nostops)

In [ ]:
# Создание словаря, где для каждого уникального слова из списка присваивается уникальный идентификатор
id2word = corpora.Dictionary(data_words_nostops)
# Преобразование списка слов в корпус, который представляет каждый документ как bow
corpus = [id2word.doc2bow(review) for review in data_words_nostops]

In [ ]:
%time find_model(1, 6, data_words_nostops)

 20%|██        | 1/5 [02:43<10:53, 163.50s/it]

1: Perplexity = -9.183. Coherence Score = 0.823


 40%|████      | 2/5 [06:18<09:40, 193.58s/it]

2: Perplexity = -9.4. Coherence Score = 0.822


 60%|██████    | 3/5 [10:26<07:16, 218.43s/it]

3: Perplexity = -9.627. Coherence Score = 0.817


 80%|████████  | 4/5 [15:06<04:02, 242.94s/it]

4: Perplexity = -9.848. Coherence Score = 0.813


100%|██████████| 5/5 [20:22<00:00, 244.41s/it]

5: Perplexity = -10.066. Coherence Score = 0.813


Perplexity of 6 themes: -10.066
Coherence Score of 2 themes: 0.823

CPU times: user 20min 8s, sys: 4.91 s, total: 20min 13s
Wall time: 20min 22s


Можно заметить, что значение когерентности осталось неизменным на 4 и 5 темах, однако на 5 значение перплексии уменьшается. Чем меньше ее (перплексии) значение, тем лучше, поэтому посмотрим визуализацию 5 тем:

In [ ]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                            id2word=id2word,
                                            num_topics=5,
                                            random_state=100,
                                            update_every=1,
                                            chunksize=100,
                                            passes=10,
                                            alpha='auto',
                                            per_word_topics=True)

In [ ]:
pprint(lda_model.print_topics())

[(0,
  '0.017*"account" + 0.015*"website" + 0.012*"time" + 0.012*"number" + '
  '0.011*"information" + 0.011*"online" + 0.010*"today" + 0.009*"right" + '
  '0.008*"month" + 0.008*"invest"'),
 (1,
  '0.025*"please" + 0.017*"people" + 0.017*"rate" + 0.017*"contact" + '
  '0.016*"message" + 0.016*"money" + 0.014*"know" + 0.014*"make" + '
  '0.013*"earn" + 0.013*"mail"'),
 (2,
  '0.027*"company" + 0.022*"video" + 0.018*"like" + 0.016*"email" + '
  '0.014*"visit" + 0.014*"best" + 0.014*"day" + 0.014*"http" + 0.013*"country" '
  '+ 0.013*"also"'),
 (3,
  '0.018*"link" + 0.016*"call" + 0.015*"group" + 0.014*"business" + '
  '0.014*"hyperlink" + 0.013*"profit" + 0.013*"life" + 0.013*"rs" + '
  '0.012*"available" + 0.012*"join"'),
 (4,
  '0.031*"free" + 0.023*"click" + 0.017*"price" + 0.015*"offer" + 0.013*"site" '
  '+ 0.011*"would" + 0.010*"take" + 0.009*"hour" + 0.009*"every" + '
  '0.009*"world"')]


In [ ]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.263271  0.008164       1        1  24.355881
0     -0.076135  0.234999       2        1  22.259845
4     -0.053281 -0.052475       3        1  19.638639
1     -0.072224 -0.112722       4        1  18.017075
2     -0.061630 -0.077967       5        1  15.728561, topic_info=          Term        Freq       Total Category  logprob  loglift
5         free  783.000000  783.000000  Default  30.0000  30.0000
206    company  549.000000  549.000000  Default  29.0000  29.0000
234     please  564.000000  564.000000  Default  28.0000  28.0000
39       click  586.000000  586.000000  Default  27.0000  27.0000
434      video  442.000000  442.000000  Default  26.0000  26.0000
...        ...         ...         ...      ...      ...      ...
2504   𝐭𝐫𝐚𝐝𝐢𝐧𝐠  139.389336  140.276084   Topic5  -4.9658   1.8434
537       even  119.038655  119.813242   Topic5  -5.1236   1.8432
1578  telegram  115.613975  116.396777   Topic5  -5.1528   1.8429
506      inbox  114.631391  115.428770   Topic5  -5.1613   1.8428
451      since  117.373522  118.204044   Topic5  -5.1377   1.8426

[185 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
304       2  0.999384    account
629       4  0.992164    address
3195      1  0.994985      adobe
6         5  0.995683       also
733       1  0.996826  available
...     ...       ...        ...
1224      3  0.988991  worldwide
704       2  0.990726      worth
152       3  0.996319      would
208       4  0.996673       year
2504      5  0.990903    𝐭𝐫𝐚𝐝𝐢𝐧𝐠

[159 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 1, 5, 2, 3])

Сгруппированные в одном квадранте круги 3-5 говорят о том, что тем слишком много. Попробуем сократить кол-во тем до 3 и посмотрим на слова, которые модель отнесла к той или иной теме:

In [ ]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                            id2word=id2word,
                                            num_topics=3,
                                            random_state=100,
                                            update_every=1,
                                            chunksize=100,
                                            passes=10,
                                            alpha='auto',
                                            per_word_topics=True)

pprint(lda_model.print_topics())

[(0,
  '0.013*"link" + 0.011*"video" + 0.010*"group" + 0.009*"life" + '
  '0.009*"profit" + 0.009*"website" + 0.009*"hyperlink" + 0.009*"rs" + '
  '0.009*"available" + 0.008*"join"'),
 (1,
  '0.016*"free" + 0.012*"company" + 0.012*"please" + 0.011*"call" + '
  '0.009*"rate" + 0.008*"contact" + 0.008*"message" + 0.008*"money" + '
  '0.007*"know" + 0.007*"make"'),
 (2,
  '0.013*"click" + 0.011*"account" + 0.010*"business" + 0.009*"people" + '
  '0.009*"price" + 0.008*"like" + 0.008*"offer" + 0.007*"investment" + '
  '0.007*"number" + 0.007*"online"')]


Таким образом, среди сообщений, которые были помечены как спам, хорошо выделяются три темы. Можно предположить, что первый тип сообщений связан с просьбами перейти на сайт, второй - с заработком денег, третий - с приобретением чего-либо

### Non-spam themes

In [ ]:
bigram = gensim.models.Phrases(non_spam_tokens_flat, min_count=3, threshold=100)
bigram_mod = gensim.models.phrases.Phraser(bigram)

In [ ]:
data_words_nostops = remove_stopwords(non_spam_tokens_flat)
data_words_bigrams = make_bigrams(data_words_nostops)

In [ ]:
id2word = corpora.Dictionary(data_words_nostops)
corpus = [id2word.doc2bow(review) for review in data_words_nostops]

In [ ]:
%time find_model(1, 6, data_words_nostops)

 20%|██        | 1/5 [05:41<22:46, 341.71s/it]

2: Perplexity = -8.84. Coherence Score = 0.834


 40%|████      | 2/5 [13:13<20:19, 406.57s/it]

3: Perplexity = -8.983. Coherence Score = 0.824


 60%|██████    | 3/5 [22:01<15:23, 461.96s/it]

4: Perplexity = -9.13. Coherence Score = 0.819


 80%|████████  | 4/5 [32:03<08:37, 517.29s/it]

5: Perplexity = -9.275. Coherence Score = 0.815


100%|██████████| 5/5 [43:28<00:00, 521.65s/it]

6: Perplexity = -9.418. Coherence Score = 0.811


Perplexity of 5 themes: -9.418
Coherence Score of 0 themes: 0.5

CPU times: user 43min 4s, sys: 13.3 s, total: 43min 17s
Wall time: 43min 28s


In [ ]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                            id2word=id2word,
                                            num_topics=5,
                                            random_state=100,
                                            update_every=1,
                                            chunksize=100,
                                            passes=10,
                                            alpha='auto',
                                            per_word_topics=True)

In [ ]:
pprint(lda_model.print_topics())

[(0,
  '0.048*"subject" + 0.033*"thanks" + 0.025*"date" + 0.023*"wrote" + '
  '0.022*"could" + 0.019*"want" + 0.016*"hope" + 0.012*"information" + '
  '0.012*"user" + 0.012*"version"'),
 (1,
  '0.032*"like" + 0.025*"call" + 0.025*"list" + 0.024*"think" + 0.018*"email" '
  '+ 0.018*"research" + 0.017*"something" + 0.017*"week" + 0.015*"phone" + '
  '0.015*"presentation"'),
 (2,
  '0.019*"kaminski" + 0.011*"take" + 0.011*"still" + 0.010*"good" + '
  '0.009*"year" + 0.009*"back" + 0.008*"company" + 0.008*"system" + '
  '0.008*"energy" + 0.007*"dear"'),
 (3,
  '0.081*"enron" + 0.029*"would" + 0.015*"message" + 0.013*"make" + '
  '0.013*"meeting" + 0.013*"well" + 0.011*"also" + 0.011*"change" + '
  '0.010*"look" + 0.009*"last"'),
 (4,
  '0.055*"vince" + 0.035*"please" + 0.026*"know" + 0.022*"time" + 0.016*"need" '
  '+ 0.014*"group" + 0.012*"shirley" + 0.012*"model" + 0.011*"mail" + '
  '0.011*"people"')]


In [ ]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5      0.388963  0.045121       1        1  67.072326
1     -0.031138 -0.289302       2        1  10.591806
2     -0.061133 -0.045516       3        1   8.461386
4     -0.090060  0.046624       4        1   5.683540
3     -0.102166  0.108784       5        1   4.933688
0     -0.104466  0.134288       6        1   3.257254, topic_info=           Term         Freq        Total Category  logprob  loglift
80        enron  4712.000000  4712.000000  Default  30.0000  30.0000
13        vince  3399.000000  3399.000000  Default  29.0000  29.0000
21       please  2137.000000  2137.000000  Default  28.0000  28.0000
20      subject  1366.000000  1366.000000  Default  27.0000  27.0000
18     kaminski  1552.000000  1552.000000  Default  26.0000  26.0000
...         ...          ...          ...      ...      ...      ...
26197    msgbuf     1.682396    55.636873   Topic6  -8.6701  -0.0743
6886       ella     2.114890   180.301324   Topic6  -8.4413  -1.0213
2527     yannis     1.872622   128.532734   Topic6  -8.5630  -0.8046
9511     bridge     1.846185   138.079101   Topic6  -8.5772  -0.8904
7558      marie     1.840535   164.118063   Topic6  -8.5802  -1.0662

[327 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
424       4  0.991952       added
202       6  0.988900  additional
4459      1  0.437644       adobe
4459      2  0.185667       adobe
4459      3  0.185667       adobe
...     ...       ...         ...
2527      3  0.178943      yannis
2527      4  0.085581      yannis
2527      5  0.046681      yannis
2527      6  0.015560      yannis
903       1  0.998721        year

[418 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 2, 3, 5, 4, 1])

Здесь можно увидеть наиболее распростраенную тему, которая, с одной стороны, не имеет каких-то точек соприкосноения, но, с другой стороны, можно предположить, что эти сообщения имеют как рабочий характер (program, system, change), так и личный (dear). Поэтому эта тема и яляется такой большой.

# Classification

In [54]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score, accuracy_score

### TF-IDF

In [55]:
df_test = pd.read_csv(PATH_test)
df_train = pd.read_csv(PATH_train)
# Заменяем текстовые метки на числовые
df_train['text_type'] = df_train['text_type'].map({'ham': 0, 'spam': 1})
# Разделяем таргет
X_train = df_train['text'].values
y_train = df_train['text_type'].values
# Преобразование в числовые векторы при помощи TF-IDF
vectorizer = TfidfVectorizer()
X_train_vectors = vectorizer.fit_transform(X_train)
# Разделение на обучающую и валидационную выборки
X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(
    X_train_vectors, y_train, test_size=0.2, random_state=42
)
# Создание и обучение модели
rf_classifier = RandomForestClassifier(random_state=42)
rf_classifier.fit(X_train_split, y_train_split)
# Оценка модели на валидационном наборе
y_val_pred = rf_classifier.predict(X_val_split)
print(classification_report(y_val_split, y_val_pred))

y_val_probs = rf_classifier.predict_proba(X_val_split)[:, 1]
# Вычисление ROC-AUC score на валидационной выборке
roc_auc_TfIDF = roc_auc_score(y_val_split, y_val_probs)
print("ROC-AUC Tf-IDF:", round(roc_auc_TfIDF, 3))
# Применение обученной модели для предсказания на df_test
X_df_test = vectorizer.transform(df_test['text'].values)
df_test['text_type_predicted'] = rf_classifier.predict(X_df_test)

              precision    recall  f1-score   support

           0       0.91      1.00      0.95      2321
           1       0.99      0.75      0.86       935

    accuracy                           0.93      3256
   macro avg       0.95      0.88      0.90      3256
weighted avg       0.93      0.93      0.92      3256

ROC-AUC Tf-IDF: 0.985


In [99]:
df_test.head()

,text,text_type_predicted
0,j jim whitehead ejw cse ucsc edu writes j you ...,0
1,original message from bitbitch magnesium net p...,0
2,java for managers vince durasoft who just taug...,0
3,there is a youtuber name saiman says,0
4,underpriced issue with high return on equity t...,1


### CountVector

In [56]:
df_test = pd.read_csv(PATH_test)
df_train = pd.read_csv(PATH_train)

df_train['text_type'] = df_train['text_type'].map({'ham': 0, 'spam': 1})
X_train = df_train['text'].values
y_train = df_train['text_type'].values
# Преобразование в числовые векторы при помощи bow
vectorizer = CountVectorizer()
X_train_vectors = vectorizer.fit_transform(X_train)

X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(
    X_train_vectors, y_train, test_size=0.2, random_state=42
)

rf_classifier = RandomForestClassifier(random_state=42)
rf_classifier.fit(X_train_split, y_train_split)

y_val_pred = rf_classifier.predict(X_val_split)
print(classification_report(y_val_split, y_val_pred))

y_val_probs = rf_classifier.predict_proba(X_val_split)[:, 1]

roc_auc_BOW = roc_auc_score(y_val_split, y_val_probs)
print("ROC-AUC Score на валидационной выборке:", round(roc_auc_BOW, 3))

X_df_test = vectorizer.transform(df_test['text'].values)
df_test['text_type_predicted'] = rf_classifier.predict(X_df_test)

              precision    recall  f1-score   support

           0       0.91      1.00      0.95      2321
           1       0.99      0.75      0.85       935

    accuracy                           0.92      3256
   macro avg       0.95      0.87      0.90      3256
weighted avg       0.93      0.92      0.92      3256

ROC-AUC Score на валидационной выборке: 0.984


In [101]:
df_test.head()

,text,text_type_predicted
0,j jim whitehead ejw cse ucsc edu writes j you ...,0
1,original message from bitbitch magnesium net p...,0
2,java for managers vince durasoft who just taug...,0
3,there is a youtuber name saiman says,0
4,underpriced issue with high return on equity t...,1


### word2vec

In [94]:
from gensim.models.word2vec import Word2Vec
import numpy as np

In [95]:
def vectorize(text: str, model: gensim.models.Word2Vec) -> np.ndarray:
    """
      Функция преобразования текста в вектор, используя предобученную модель word2vec

      Args:

          text - текст, который необходимо преобразовать в вектор

          model - предобученная модель word2vec

      Returns:

          Усредненный вектор по всем словам
    """
    # Инициализация нулевого вектора
    vector = np.zeros(model.vector_size)
    num_words = 0

    for word in text.split():
        if word in model.wv:
            # Сложение векторного представления с текущим значением
            vector += model.wv[word]
            num_words += 1
    if num_words:
        # Усреднение вектора
        vector /= num_words

    return vector

In [96]:
df_test = pd.read_csv(PATH_test)
df_train = pd.read_csv(PATH_train)

In [97]:
df_train_tokenized = df_train.copy()
df_train_tokenized['text'] = df_train_tokenized['text'].apply(preprocess)
df_train_tokenized['text'] = df_train_tokenized['text'].apply(lambda x: ' '.join(x))

df_test_tokenized = df_test.copy()
df_test_tokenized['text'] = df_test_tokenized['text'].apply(preprocess)
df_test_tokenized['text'] = df_test_tokenized['text'].apply(lambda x: ' '.join(x))

In [98]:
# Создаем список для последующего обучения мдоели word2vec
sentences = [message.split() for message in df_train_tokenized['text']]
# Создание и обучение модели с параметрами
# vector_size - размерность векторов слов
# window - сколько слов используется для обучения (расстояние)
# min_count - мин. кол-во вхождений слова для его включения в словарь
word2vec_model = Word2Vec(sentences, vector_size=100, window=5, min_count=5)

In [99]:
X_train = df_train_tokenized['text'].apply(lambda x: vectorize(x, word2vec_model))
X_test = df_test_tokenized['text'].apply(lambda x: vectorize(x, word2vec_model))

In [100]:
X_train = np.stack(X_train.values)
X_test = np.stack(X_test.values)

In [101]:
y_train = df_train_tokenized['text_type'].values
classifier = RandomForestClassifier(random_state=42)
classifier.fit(X_train, y_train)
# Оценка классификатора (Для оценки ROC-AUC, нам нужны вероятности предсказаний для класса спам)
y_train_pred = classifier.predict_proba(X_train)[:, 1]
roc_auc_vec = roc_auc_score(y_train, y_train_pred)
print("ROC-AUC на обучающем наборе:", roc_auc_vec)
# Классификация текстов из df_test
y_test_pred = classifier.predict_proba(X_test)[:, 1]
# Добавление результатов в df_test для последующего анализа
df_test['text_type_predicted_proba'] = y_test_pred

ROC-AUC на обучающем наборе: 0.9984202807604489


### PyTorch

In [78]:
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import numpy as np

In [79]:
# Определение модели
class SpamClassifier(nn.Module):
    def __init__(self):
        super(SpamClassifier, self).__init__()
        self.fc1 = nn.Linear(X_train.shape[1], 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 2)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [80]:
df_test = pd.read_csv(PATH_test)
df_train = pd.read_csv(PATH_train)

vectorizer = TfidfVectorizer()

df_train['text_type'] = df_train['text_type'].map({'ham': 0, 'spam': 1}).astype(int)
df_train.dropna(subset=['text_type'], inplace=True)

X_train = vectorizer.fit_transform(df_train['text']).toarray()
y_train = df_train['text_type'].values
X_test = vectorizer.transform(df_test['text']).toarray()
# Преобразование данных
X_train_tensor = torch.FloatTensor(X_train)
y_train_tensor = torch.LongTensor(y_train)
X_test_tensor = torch.FloatTensor(X_test)

train_data = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_data, batch_size=16, shuffle=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SpamClassifier().to(device)
# Функция потерь и оптимизатор
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
# Обучение модели
num_epochs = 5

for epoch in range(num_epochs):
    model.train()

    for texts, labels in train_loader:
        texts, labels = texts.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(texts)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')
# Предсказание для тестового набора данных
model.eval()
X_test_tensor = X_test_tensor.to(device)

with torch.no_grad():
    outputs = model(X_test_tensor)
    _, predicted = torch.max(outputs.data, 1)
# Преобразование предсказания в массив
predicted_np = predicted.cpu().numpy()

Epoch 1/5, Loss: 0.02698611468076706
Epoch 2/5, Loss: 0.0006182528450153768
Epoch 3/5, Loss: 0.00044286251068115234
Epoch 4/5, Loss: 0.00019743172742892057
Epoch 5/5, Loss: 0.00018822593847289681


In [81]:
def calculate_roc_auc(model, X_test_tensor, y_test):
    model.eval()

    with torch.no_grad():
        probabilities = torch.softmax(model(X_test_tensor.to(device)), dim=1)
        predictions_prob = probabilities[:, 1]
        roc_auc = roc_auc_score(y_test.cpu(), predictions_prob.cpu())

    return roc_auc

roc_auc_pytorch = calculate_roc_auc(model, X_train_tensor, y_train_tensor)

print(f'ROC-AUC score: {roc_auc_pytorch}')

ROC-AUC score: 0.999994488202496


### LogisticRegression, Naive Bayes

In [29]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV

In [32]:
from sklearn.naive_bayes import MultinomialNB

In [45]:
df_train = pd.read_csv(PATH_train)
df_train['text_type'] = df_train['text_type'].map({'ham': 0, 'spam': 1}).astype(int)

vectorizer = CountVectorizer()

X = vectorizer.fit_transform(df_train['text'])
y = df_train['text_type']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [46]:
nb_classifier = MultinomialNB()
nb_classifier.fit(X_train, y_train)

y_scores = nb_classifier.predict_proba(X_test)
y_scores_pos = y_scores[:, 1]
roc_auc_nb = roc_auc_score(y_test, y_scores_pos)

print(f"ROC-AUC score: {roc_auc_nb}")

ROC-AUC score: 0.9662205346671981


In [47]:
model = LogisticRegression()
model.fit(X_train, y_train)

y_scores = nb_classifier.predict_proba(X_test)
y_scores_pos = y_scores[:, 1]
roc_auc_logreg = roc_auc_score(y_test, y_scores_pos)

print(f"ROC-AUC score: {roc_auc_logreg}")

ROC-AUC score: 0.9662205346671981


In [51]:
df_test = pd.read_csv(PATH_test)
df_train = pd.read_csv(PATH_train)

df_train['text_type'] = df_train['text_type'].map({'ham': 0, 'spam': 1}).astype(int)

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df_train['text'])
y = df_train['text_type']

param_grid = {
              'solver': ['newton-cg', 'lbfgs', 'liblinear'],
              'C': [0.001, 0.01, 0.1, 1, 10, 100],
              'penalty': ['l1', 'l2']
             }

grid_search = GridSearchCV(LogisticRegression(), param_grid, cv=5, n_jobs=-1)
grid_search.fit(X, y)
print(f"Best params: {grid_search.best_params_}")

best_model = grid_search.best_estimator_
y_pred = best_model.predict(X)

roc_auc_logreg_gscv = roc_auc_score(y, y_pred)
print("ROC-AUC на обучающем наборе:", roc_auc_logreg_gscv)

Best params: {'C': 1, 'penalty': 'l2', 'solver': 'liblinear'}
ROC-AUC на обучающем наборе: 0.9901797718083198


In [52]:
param_grid = {
              'C': [0.001, 0.01, 0.1, 1, 10, 100],
              'penalty': ['l1', 'l2', 'elasticnet', 'none']
             }

random_search_logreg = RandomizedSearchCV(
                            LogisticRegression(), param_grid, n_iter=100, cv=5,
                            verbose=1, random_state=42, n_jobs=-1
                            )

random_search_logreg.fit(X, y)
best_params_logreg = random_search_logreg.best_params_

print(f"Best params: {best_params_logreg}")

best_logreg_model = random_search_logreg.best_estimator_
y_pred_logreg = best_logreg_model.predict(X)
roc_auc_random = roc_auc_score(y, y_pred)
print("ROC-AUC на обучающем наборе:", roc_auc_random)

Fitting 5 folds for each of 24 candidates, totalling 120 fits
Best params: {'penalty': 'l2', 'C': 1}
ROC-AUC на обучающем наборе: 0.9901797718083198


# Conclusion

In [83]:
ra_list = [roc_auc_TfIDF, roc_auc_BOW, roc_auc_vec, roc_auc_pytorch,
           roc_auc_nb, roc_auc_logreg, roc_auc_logreg_gscv, roc_auc_random]
col_names_list = ['Tf-IDF', 'BOW', 'Word2Vec', 'PyTorch', 'NB',
                  'LogReg', 'LogRegGSCV', 'LogRegRSCV']

In [85]:
conclusion_dict = dict(zip(col_names_list, ra_list))
df_conclusion = pd.DataFrame([conclusion_dict])

In [86]:
df_conclusion

,Tf-IDF,BOW,Word2Vec,PyTorch,NB,LogReg,LogRegGSCV,LogRegRSCV
0,0.985167,0.983728,0.998414,0.999994,0.966221,0.966221,0.99018,0.99018


Как видно из приведенной сравнительной таблицы, лучшие результаты показал подход с использованием PyTorch. Этот метод улавливает контекст и порядок слов, которые игнорируются в подходах типа BOW и TF-IDF.

Подходы NB и LogReg получили самые низкие значения, так как не было предобработки, тем не менее использование GridSearchCV и RandomizedSearchCV смогло значительно улучшить показатели моделей.

Сравнивая Word2Vec и PyTorch, которые имеют крайше схожие значения, фреймворк показал лучшие значения, так как он позволяет конструировать и тренировать сложные нейронные сети, которые могут автоматически извлекать более сложные и абстрактные признаки из текста. Word2Vec же представляет слова в виде векторов, сохраняя семантические отношения, но не учитывая контекст слов в предложениях

In [109]:
df_test['predicted'] = pd.DataFrame(predicted_np)

In [112]:
df_test.head()

,text,predicted
0,j jim whitehead ejw cse ucsc edu writes j you ...,0
1,original message from bitbitch magnesium net p...,0
2,java for managers vince durasoft who just taug...,0
3,there is a youtuber name saiman says,0
4,underpriced issue with high return on equity t...,1


In [113]:
# Меняем обратно численные значения на ham/spam
df_test['predicted'] = df_test['predicted'].map({0: 'ham', 1: 'spam'})
# Переименовываем столбцы
df_test.columns = ['text', 'score']
# По условию задачи сначала идет колонка score
df_test = df_test[['score', 'text']]
df_test.to_csv('predicted_messages.csv', index=False)